In [7]:
import json
import requests
import six
import six.moves.urllib.parse as urllibparse
import pandas as pd
import matplotlib
import pyperclip as cp
from math import ceil
import heapq
import base64

%matplotlib inline
import matplotlib.pyplot as plt

In [8]:
def get_auth_code(url):
    start = url.index('?')
    end = url.index('&')
    code = url[start + 6:end]
    return code

def create_auth_header(client_id, client_secret):
    auth_header = base64.b64encode(six.text_type(client_id + ':' + client_secret).encode('ascii'))
    return {'Authorization': 'Basic %s' % auth_header.decode('ascii')}

In [9]:
username = '99e5678q6fmxtbkbeaxhn7oxt'
scope = "user-top-read user-read-recently-played playlist-modify-public user-library-read user-library-modify user-read-playback-state user-read-currently-playing user-modify-playback-state"
client_id = '66cd3e7af76e433486f0df965e20e25c'
client_secret = '152030c0f9674428b7fe98332d5cc807'
redirect_uri = 'https://example.com/callback/'
state = 'krish'

auth_code_url = "https://accounts.spotify.com/authorize"
auth_code_dict = {"client_id": client_id, "response_type": "code", "redirect_uri": redirect_uri, "state": state, "scope": scope, "show_dialog": "false"}

auth_code_params = urllibparse.urlencode(auth_code_dict)
                    
get_response = requests.get(auth_code_url, params=auth_code_params)
if get_response.status_code != 200:
    print("Error:", get_response.reason)
    
print(get_response.url)

https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fclient_id%3D66cd3e7af76e433486f0df965e20e25c%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fexample.com%252Fcallback%252F%26state%3Dkrish%26scope%3Duser-top-read%2Buser-read-recently-played%2Bplaylist-modify-public%2Buser-library-read%2Buser-library-modify%2Buser-read-playback-state%2Buser-read-currently-playing%2Buser-modify-playback-state%26show_dialog%3Dfalse


In [10]:
token_url = "https://accounts.spotify.com/api/token"
get_response_url = "https://example.com/callback/?code=AQDrq69ppsCaN8JYOre3kv1G0wVgZs1N1iQ7ZB9FuaCx6wtDoBtjrti_QyzLkInt8s39R2wbJvfJyQbPsXmP_7tpSuxaSw3GHoouzTG4hp5mWauzMxeXLb-UEbkUe-SNaf-ZbGHIRqU8T9-4K53OeoIZrRJgp7QxCqskSTtJJu7qTQetL2KbJcRym2_LhNKelG0Ot_chtq5nOJmq6P6XRzWUkiRST6CQk_X-f1_oMKbPf9wASiRGcLa4SGRAI48KhmIVGeSH9BcXBAsS68kcy5ovahmfQZmghP6ptLKulscLIYuaIeGtospS1MKMimr7zyJuxFaaG7pT4P84CMfX-8Unowsmu5QKqfVB6nmwezwoKWsP6iNPgWvQnIr8YjWWZC6FGFKphm0qLkZ4PzfuNMYz8HnEQFTm8cIanwfjSCE2HUAPF-fU-XeGdS4R2rVAxp4&state=krish"
token_data = {"redirect_uri": redirect_uri, "code": get_auth_code(get_response_url), "grant_type": "authorization_code", "scope": scope}
token_headers = create_auth_header(client_id, client_secret)

post_response = requests.post(token_url, data=token_data, headers=token_headers)

if post_response.status_code != 200:
    print("Error:", post_response.reason)

ACCESS_TOKEN = json.loads(post_response.text)["access_token"]
print(ACCESS_TOKEN)

BQAtOxp8jGhTm8lehN3fHM-2tPKivIn4uvzK-XfViAS4VKkPBcWZJMo9BNxyEbMW16XjtBPtZmbx1uedcJLH54dH-5qYB4jnhhPtOUvBdEao74EeMOS_nxYa9rS9Ge7cO4Ld3ZskYl0aXHTMCnk1tQ1Js1Qp9natOx3wZJokViF3HEdgWOgo2l9oX4DJiXflBMTPR8nsBz4gmoajCS7NCaU8Kd7Nyhj24l73Umo


In [11]:
url = "https://api.spotify.com/v1/me/tracks"
headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
tracks = requests.get(url, headers=headers)
parsed = json.loads(tracks.text)
cp.copy(tracks.text)

count_songs = parsed["total"]
print("Number of songs: {}".format(count_songs))

Number of songs: 528


In [12]:
all_songs = []

for i in range(int(ceil(count_songs/50.0))):
    offset = 50*i
    url = "https://api.spotify.com/v1/me/tracks?limit=50&offset={}".format(offset)
    track_headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
    tracks = requests.get(url, headers=track_headers)
    cp.copy(tracks.text)
    parsed = json.loads(tracks.text)

    all_songs.extend(parsed['items'])
    
try:
    if (len(all_songs) == count_songs):
        print("No responses failed.")
except:
    print("Response from API failed.")
    
print("Number of gathered songs: {}".format(len(all_songs)))

No responses failed.
Number of gathered songs: 528


In [13]:
songs_all = [(all_songs[i]['track']['popularity'], all_songs[i]['track']['name']) for i in range(len(all_songs))]
songs_under_5 = [(all_songs[i]['track']['popularity'], all_songs[i]['track']['name']) for i in range(len(all_songs)) if all_songs[i]['track']['duration_ms'] <= 300000]    
album_ids = {(all_songs[i]['track']['album']['name'], all_songs[i]['track']['album']['id']) 
             for i in range(len(all_songs))}

print("Total songs:", len(songs_all))
print("Total songs under 5 minutes:", len(songs_under_5))

Total songs: 528
Total songs under 5 minutes: 516


In [14]:
largest = heapq.nsmallest(10, songs_under_5)
largest = [(key, value) for value, key in largest]
for song in largest:
    print(song)

('Kyrie', 0)
('Chanel (feat. Swae Lee & Pharrell) - From Jxmtro', 1)
('Clout (feat. 21 Savage)', 1)
('Diplomatic Immunity', 2)
('Pineapple (feat. Gucci Mane & Quavo)', 2)
('Tired of Love', 2)
('No More (feat. Travis Scott, Kodak Black & 21 Savage)', 3)
('Nonchalant', 3)
('Bounce Out With That', 4)
('Dreamcatcher (feat. Swae Lee & Travis Scott)', 4)


In [15]:
count = 0
for item in album_ids:
    if (count < 10):
        print(item)
        count += 1
    else:
        break

('7 EP', '4IRiXE5NROxknUSAUSjMoO')
('100 Shooters (feat. Meek Mill & Doe Boy)', '5M1abRnhUWaLZdgd29gQ6M')
('Va Bene (Extrait de la BOF de "Taxi 5")', '4dJTCpetg56FrxROQ4dKvv')
('More Life', '4dvkEfxroInqojJWP06R2V')
('Freaky', '3GZxORgeTp5jEIUPUNBu7F')
('No Complaints', '4VXEIoFGEjhFMW7wYAvWSf')
('Scorpion', '2o9McLtDM7mbODV7yZF2mc')
('Pure Water (with Migos)', '1ypNy6o0C2ZbuuPPYkavVl')
('Wow.', '5Dcz8PZHKuvTyjbqEPL5VM')
('Show U Off (feat. Lil Uzi Vert)', '2WyjmS6PEAE2V9oh5So588')


In [16]:
from __future__ import division

def get_top_music(type="artists", limit=20, time_range="medium_term", offset="0", access_token=ACCESS_TOKEN):
    url = "https://api.spotify.com/v1/me/top/"
    if (type == "artists"):
        url += "artists"
    else:
        url += "tracks"
    
    query_params = {"limit": str(limit), "offset": offset, "time_range": time_range}
    headers = {'Authorization': "Bearer {}".format(access_token)}
    
    top_tracks_and_artists = requests.get(url=url, params=query_params, headers=headers)
    parsed = json.loads(top_tracks_and_artists.text)
    items = parsed['items']

    top_music = []
    
    if (type == "artists"):
        for i in range(len(items)):
            top_music.append(["Rank: {}".format(str(i+1)), items[i]['name'], int(items[i]['followers']['total']), items[i]['images'][0]['url']])
    else:
        for i in range(len(items)):
            current_item = items[i]
            artists = current_item['artists']
            current_album = current_item['album']
            length_of_song = str((current_item['duration_ms'] // 1000) // 60) + "m " + str((current_item['duration_ms'] // 1000) % 60) + "s"
            top_music.append(["Rank: {}".format(str(i+1)), current_item['name'], 
                               [artists[j]['name'] for j in range(len(artists))], 
                               current_album['name'], length_of_song, current_album['release_date'], current_item['uri']])
    return top_music
    

In [17]:
def get_top_genres(artist_limit=50, genre_limit=10, time_range="medium_term", offset="0", access_token=ACCESS_TOKEN):
    url = "https://api.spotify.com/v1/me/top/artists"
    all_genres = {}
    
    query_params = {"limit": str(artist_limit), "offset": offset, "time_range": time_range}
    headers = {'Authorization': "Bearer {}".format(access_token)}
    
    top_tracks_and_artists = requests.get(url=url, params=query_params, headers=headers)
    parsed = json.loads(top_tracks_and_artists.text)
    items = parsed['items']
    
    for i in range(len(items)):
        genres = items[i]['genres']
        for genre in genres:
            if genre not in all_genres.keys():
                all_genres[genre] = 1
            else:
                all_genres[genre] += 1
                
    heap = [[value, key.upper()] for key, value in all_genres.items()]
    largest = heapq.nlargest(genre_limit, heap)
        
    return largest

In [19]:
for artist in get_top_music(type="artists", limit=25, time_range="long_term"):
    print(artist)

['Rank: 1', 'Pi’erre Bourne', 90126, 'https://i.scdn.co/image/52492b2ebf7587a21678f6aa3cb8ff5428391235']
['Rank: 2', 'Ed Sheeran', 48880173, 'https://i.scdn.co/image/f55cab0739390cf3b2c2f773b9c779b2f0ae8a99']
['Rank: 3', '21 Savage', 6498947, 'https://i.scdn.co/image/cfd6e47fbe4446750ec209dfa37bc919cb55c86f']
['Rank: 4', 'NAV', 1501935, 'https://i.scdn.co/image/819c7e7c7126e367b35ff23e0653b1779069128f']
['Rank: 5', 'DJ Snake', 5362404, 'https://i.scdn.co/image/4c684a7c0a70d1f506595739c14f8a76ebef6687']
['Rank: 6', 'Trippie Redd', 3279808, 'https://i.scdn.co/image/18d243ea2cd23ad420c1afc8c6bb9f92e8fa5885']
['Rank: 7', 'iann dior', 99841, 'https://i.scdn.co/image/99aed49720354b73035cb96502fdc092f5f385a1']
['Rank: 8', 'Future', 7959083, 'https://i.scdn.co/image/d5ac5dc59664cf7c64795271602316dde494a6d1']
['Rank: 9', 'Lil Uzi Vert', 6746039, 'https://i.scdn.co/image/a6dd2fabeba127e6eea02bd234545de91c44e604']
['Rank: 10', 'Travis Scott', 6222884, 'https://i.scdn.co/image/dc5eba5e032c2e5bc4d4

In [27]:
for track in get_top_music(type="tracks", limit=25, time_range="short_term"):
    print(track[0], track[1], "|", track[3])

Rank: 1 Negative Energy | A Love Letter To You 3
Rank: 2 Maps | V
Rank: 3 Murda (feat. Trippie Redd) | Decided
Rank: 4 gone girl | gone girl
Rank: 5 1000 Nights (feat. Meek Mill & A Boogie Wit da Hoodie) | No.6 Collaborations Project
Rank: 6 Goin Baby | Baby On Baby
Rank: 7 ICE (feat. Gunna & Lil Baby) | Delusions of Grandeur
Rank: 8 Noticed | Northsbest (Extended)
Rank: 9 Elisa's Theme | The Shape Of Water (Original Motion Picture Soundtrack)
Rank: 10 Fall Threw (feat. Young Thug & Gunna) | The World Is Yours 2
Rank: 11 Same Yung Nigga (feat. Playboi Carti) | Drip or Drown 2
Rank: 12 Tum Hi Ho | Aashiqui 2
Rank: 13 Paradise | Luv Is Rage
Rank: 14 Bossanova | Bossanova
Rank: 15 Feeling | Death Race For Love
Rank: 16 Ice Tray | Quality Control: Control The Streets Volume 1
Rank: 17 Call You Mine | World War Joy
Rank: 18 The Theory of Everything | The Theory of Everything (Original Motion Picture Soundtrack)
Rank: 19 Suge | Baby On Baby
Rank: 20 CUDDLE MY WRIST | BEASTMODE 2
Rank: 21 Jui

In [28]:
for genre in get_top_genres(genre_limit=20, time_range="long_term"):
    print(genre)

[27, 'RAP']
[25, 'POP']
[22, 'TRAP MUSIC']
[18, 'POP RAP']
[12, 'HIP HOP']
[11, 'SOUTHERN HIP HOP']
[9, 'ATL HIP HOP']
[5, 'UNDERGROUND HIP HOP']
[5, 'CANADIAN HIP HOP']
[4, 'TORONTO RAP']
[3, 'VAPOR TRAP']
[3, 'EDM']
[3, 'DANCE POP']
[2, 'UK POP']
[2, 'TROPICAL HOUSE']
[2, 'TRAP SOUL']
[2, 'R&B']
[2, 'CANADIAN POP']
[2, 'CALI RAP']
[2, 'ATL TRAP']


In [29]:
user_id = "99e5678q6fmxtbkbeaxhn7oxt"
playlists_url = "https://api.spotify.com/v1/users/{}/playlists".format(user_id)

body_params = {"name": "code", "public": "true", "collaborative": "false", "description": "finally, a playlist with songs you never have to skip"}
header_fields = {'Authorization': "Bearer {}".format(ACCESS_TOKEN), "Content-Type": "application/json"}

new_playlist = requests.post(playlists_url, data=json.dumps(body_params), headers=header_fields)
new_playlist.ok

True

In [30]:
parsed = json.loads(new_playlist.text)
playlist_id = parsed['id']
update_playlist_url = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist_id)

top_tracks = get_top_music(type="tracks", limit=50, offset="0", time_range="medium_term")
top_tracks.pop()
top_tracks.extend(get_top_music(type="tracks", limit=26, offset="49", time_range="medium_term"))

uris = [track[6] for track in top_tracks]

body_params = {"uris": uris}
header_fields = {'Authorization': "Bearer {}".format(ACCESS_TOKEN), "Content-Type": "application/json"}

updated_playlist = requests.post(update_playlist_url, data=json.dumps(body_params), headers=header_fields)
updated_playlist.ok

True